# TEST

In [ ]:
import torch
import os
import re
import pandas as pd
import numpy as np
import glob
import json
import pickle as pkl
from pathlib import Path
from tqdm.auto import tqdm

basepath = "/home/abetatos/Escritorio/TFM_Dataset/"
model_basepath = "/home/abetatos/Escritorio/TFM_models/"

date = "20230523"
config = "256pixel_4ro"
model_id = "1684604492"

size = int(re.findall("\d+", config)[0])
dataset_path = f"/home/abetatos/Escritorio/TFM_Dataset/Dataset_{date}/{config}/"

pred_model = "1684923533_model_57_012345678"

df = pd.DataFrame([[f] for f in glob.glob(dataset_path + "Data/**/*.pickle", recursive=True)], columns=['filename'])
df['place'] = df['filename'].apply(lambda x: Path(x).name.split("__")[0])
df['col_off'] = df['filename'].apply(lambda x: Path(x).name.split("__")[-2]).astype(int)
df['row_off'] = df['filename'].apply(lambda x: Path(x).name.split("__")[-1].replace(".pickle", "")).astype(int)

for place, place_df in df.groupby("place"):
    print(place)
    place_df.to_csv("tmp_df.csv")
    
    max_col_off = place_df['row_off'].max() + size
    max_row_off = place_df['col_off'].max() + size

    mapping = np.zeros((max_col_off, max_row_off))
    counter = np.zeros((max_col_off, max_row_off))
    for index, name in place_df['filename'].items(): 
        with open(name, "rb") as f: 
            pred = pkl.load(f)[-1]
        row_off = place_df['row_off'].loc[index]
        col_off = place_df['col_off'].loc[index]

        mapping[row_off:row_off+size, col_off:col_off+size] += pred
        counter[row_off:row_off+size, col_off:col_off+size] += np.where(pred != -9999., 1, 0)

    outpath = f"/home/abetatos/Escritorio/TFM_Dataset/5.Preds/prueba/{place}/"
    os.makedirs(outpath, exist_ok=True)
    with open(f"{outpath}HS_pred.pickle", "wb") as src: 
        norm_mapping = np.where(counter != 0, np.divide(mapping, counter + 1e-10), 0)
        pkl.dump(norm_mapping, src)
    break

HS_Davos_20170316_2m_epsg2056


In [ ]:
modelname = "1684923533_model_57_012345678"

pred_path = f"/home/abetatos/Escritorio/TFM_Dataset/5.Preds/prueba/"
hs_path = "/home/abetatos/Escritorio/TFM_Dataset/4.DatasetPost/HS/"

In [ ]:
for filename in os.listdir(pred_path): 
    print(filename)
    with open(f"{pred_path}/{filename}/HS_pred.pickle", "rb") as f: 
        mapping = pkl.load(f)
        shape = mapping.shape

    with rio.open(f"{hs_path}/{filename}/HS.tif", "r") as f: 
        profile = f.profile
        arr = f.read().squeeze()
        
    new_mapping = np.ones_like(arr)*-9999.
    
    new_mapping[0:shape[0], 0:shape[1]] = mapping

    with rio.open(f"{pred_path}/{filename}/HS_pred.tif", "w", **profile) as f:
        f.write(new_mapping, 1)

    with rio.open(f"{pred_path}/{filename}/HS_diff.tif", "w", **profile) as f:
        diff_array = np.where((arr != -9999.) & (new_mapping != -9999.), abs(arr-new_mapping), -9999.)
        f.write(diff_array, 1)

HS_Davos_20170316_2m_epsg2056
